# 深度可分离卷积

卷积神经网络进化版，功能一致，但是速度更快、表示效率更高

卷积神经网络由卷积层和池化层组成，一般一个卷积层配一个池化层，多个卷积层配一个池化层也比较常见，局部相关，

一般应用场景：
- 2D卷积神经网络：图像识别
- 1D卷积神经网络：智能翻译

## 1. 导入包，并打印版本信息

In [1]:
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

models = tf.keras.models
layers = tf.keras.layers
activations = tf.keras.activations
mnist = tf.keras.datasets.mnist
to_categorical = tf.keras.utils.to_categorical
optimizers = tf.keras.optimizers
losses = tf.keras.losses

print('tensorflow: ' + str(tf.__version__))
print('matplotlib: ' + str(matplotlib.__version__))
print('numpy: ' + str(np.__version__))

tensorflow: 1.14.0
matplotlib: 3.1.0
numpy: 1.16.4


## 2. 准备样本数据

In [2]:
(train_images, train_labels_original), (test_images, test_labels_original) = mnist.load_data()

train_images = train_images.reshape((len(train_images), 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((len(test_images), 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)

val_samples = 30000

data_train_images = train_images[val_samples:]
data_train_labels = train_labels[val_samples:]

val_images = train_images[:val_samples]
val_labels = train_labels[:val_samples]

## 3. 构建模型

In [3]:
def build_model_1():
    model = models.Sequential()
    model.add(layers.SeparableConv2D(32, (3, 3), activation=activations.relu, input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.SeparableConv2D(64, (3, 3), activation=activations.relu))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation=activations.relu))
    model.add(layers.Dense(10, activation=activations.softmax))

    return model


def build_model_2():
    model = models.Sequential()
    model.add(layers.Conv2D(16, (5, 5), padding='same', input_shape=(28, 28, 1), activation=activations.relu))
    model.add(layers.MaxPool2D((2, 1)))
    model.add(layers.Conv2D(36, (5, 5), padding='same', activation=activations.relu))
    model.add(layers.MaxPool2D((2, 1)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation=activations.relu))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation=activations.softmax))

    return model

model = build_model_1()

model.summary()

W0729 01:36:05.073590 140448259446592 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 26, 26, 32)        73        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 11, 11, 64)        2400      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                102464    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6

## 4. 编译模型

In [4]:
model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.categorical_crossentropy,
    metrics=['accuracy', 'mae', 'mse']
)

## 5. 训练模型

In [ ]:
history = model.fit(
    data_train_images,
    data_train_labels,
    epochs=10,
    batch_size=64,
    validation_data=(val_images, val_labels)
)

Train on 30000 samples, validate on 30000 samples
Epoch 1/10
30000/30000 [==============================] - 17s 556us/sample - loss: 0.4364 - acc: 0.8726 - mean_absolute_error: 0.0423 - mean_squared_error: 0.0192 - val_loss: 0.1747 - val_acc: 0.9466 - val_mean_absolute_error: 0.0179 - val_mean_squared_error: 0.0081
Epoch 2/10
30000/30000 [==============================] - 17s 555us/sample - loss: 0.1347 - acc: 0.9574 - mean_absolute_error: 0.0136 - mean_squared_error: 0.0064 - val_loss: 0.1355 - val_acc: 0.9565 - val_mean_absolute_error: 0.0129 - val_mean_squared_error: 0.0066
Epoch 3/10
30000/30000 [==============================] - 17s 551us/sample - loss: 0.1056 - acc: 0.9662 - mean_absolute_error: 0.0107 - mean_squared_error: 0.0051 - val_loss: 0.0951 - val_acc: 0.9710 - val_mean_absolute_error: 0.0097 - val_mean_squared_error: 0.0045
Epoch 4/10
30000/30000 [==============================] - 17s 551us/sample - loss: 0.0864 - acc: 0.9717 - mean_absolute_error: 0.0089 - mean_squared_

## 6. 评估模型

In [ ]:
model.evaluate(test_images, test_labels)

## 7. 预测结果

In [ ]:
x_samples = test_images[:100]
y_samples = test_labels_original[:100]
predications = model.predict(x_samples)

## 8. 使用图形显示100个结果

In [ ]:
plt.figure(figsize=(25, 25))
for i in range(100):
    plt.subplot(10, 10, i + 1)
    plt.xticks([])
    plt.yticks([])    
    plt.imshow(x_samples[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.grid(True)
    if np.argmax(predications[i]) != test_labels_original[i]:       
        plt.xlabel('predict:' + str(np.argmax(predications[i])) + ', actual:' + str(y_samples[i]), color='red')
    else:
        plt.xlabel('predict:' + str(np.argmax(predications[i])) + ', actual:' + str(y_samples[i]))
plt.show()